In [2]:
# import libaries
import numpy as np
import pandas as pd
import nltk, pprint
import matplotlib.pyplot as plt
import random

import gzip, os, pickle # gzip for reading the gz files, pickle to save/dump trained model 
import _pickle as cPickle

import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV

# supress warnings
import warnings
warnings.filterwarnings('ignore')

C:\Users\ab9bh\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\ab9bh\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [27]:
# reading the Treebank tagged sentences
wsj = list(nltk.corpus.treebank.tagged_sents())

In [30]:
# Splitting into train and test
random.seed(27)
train_set, test_set = train_test_split(wsj,test_size=0.05)

In [31]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95647

In [45]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
tokens[:10]

['By',
 'Tuesday',
 'night',
 ',',
 'television',
 'stations',
 'were',
 'carrying',
 'new',
 'ads']

In [46]:
# vocabulary
V = set(tokens)
print(len(V))

12072


In [47]:
# number of tags
T = set([pair[1] for pair in train_tagged_words])
len(T)

46

### Emission Probabilities

In [48]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

In [49]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

### Transition Probabilities

In [50]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [51]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [52]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

## Viterbi Algorithm

In [53]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))



In [54]:
# Running on entire test dataset would take more than 3-4hrs. 
# Let's test our Viterbi algorithm on a few sample sentences of test dataset

random.seed(27)

# choose random 5 sents
rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
test_run[0]

[('Since', 'IN'),
 ('the', 'DT'),
 ('new', 'JJ'),
 ('park', 'NN'),
 ('will', 'MD'),
 ('have', 'VB'),
 ('only', 'RB'),
 ('1,500', 'CD'),
 ('spaces', 'NNS'),
 (',', ','),
 ('Mr.', 'NNP'),
 ('Christopher', 'NNP'),
 ('thinks', 'VBZ'),
 ('0', '-NONE-'),
 ('backers', 'NNS'),
 ('are', 'VBP'),
 ('playing', 'VBG'),
 ('some', 'DT'),
 ('fiscal', 'JJ'),
 ('``', '``'),
 ('games', 'NNS'),
 ("''", "''"),
 ('of', 'IN'),
 ('their', 'PRP$'),
 ('own', 'JJ'),
 ('with', 'IN'),
 ('the', 'DT'),
 ('voters', 'NNS'),
 ('.', '.')]

In [62]:
test1 = 'Android is a mobile operating system developed by Google.'

In [63]:
# tagging the test sentences
start = time.time()
#tagged_seq = Viterbi(test_tagged_words)
tagged_seq = Viterbi(test1)
end = time.time()
difference = end-start

In [56]:
print("Time taken in seconds: ", difference)
print(tagged_seq)
#print(test_run_base)

Time taken in seconds:  69.37051057815552
[('Since', 'IN'), ('the', 'DT'), ('new', 'JJ'), ('park', 'NN'), ('will', 'MD'), ('have', 'VB'), ('only', 'RB'), ('1,500', 'CD'), ('spaces', 'NNS'), (',', ','), ('Mr.', 'NNP'), ('Christopher', 'NNP'), ('thinks', 'VBZ'), ('0', '-NONE-'), ('backers', 'NNS'), ('are', 'VBP'), ('playing', 'VBG'), ('some', 'DT'), ('fiscal', 'JJ'), ('``', '``'), ('games', 'CC'), ("''", 'CC'), ('of', 'IN'), ('their', 'PRP$'), ('own', 'JJ'), ('with', 'IN'), ('the', 'DT'), ('voters', 'NNS'), ('.', '.'), ('The', 'DT'), ('financial-services', 'JJ'), ('company', 'NN'), ('will', 'MD'), ('pay', 'VB'), ('0.82', 'CD'), ('share', 'NN'), ('for', 'IN'), ('each', 'DT'), ('Williams', 'NNP'), ('share', 'NN'), ('.', '.'), ('The', 'DT'), ('forthcoming', 'CC'), ('maturity', 'NN'), ('in', 'IN'), ('November', 'NNP'), ('of', 'IN'), ('a', 'DT'), ('10-year', 'JJ'), ('Japanese', 'JJ'), ('government', 'NN'), ('yen-denominated', 'CC'), ('bond', 'NN'), ('issue', 'NN'), ('valued', 'VBN'), ('*', '-

In [58]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
accuracy

0.9

In [60]:
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

[[('``', '``'), (('games', 'CC'), ('games', 'NNS'))],
 [('games', 'NNS'), (("''", 'CC'), ("''", "''"))],
 [('The', 'DT'),
  (('financial-services', 'JJ'), ('financial-services', 'NNS'))],
 [('The', 'DT'), (('forthcoming', 'CC'), ('forthcoming', 'JJ'))],
 [('a', 'DT'), (('10-year', 'JJ'), ('10-year', 'CD'))],
 [('government', 'NN'),
  (('yen-denominated', 'CC'), ('yen-denominated', 'JJ'))],
 [('at', 'IN'), (('about', 'IN'), ('about', 'RB'))],
 [('investors', 'NNS'), (('redeeming', 'CC'), ('redeeming', 'VBG'))],
 [('figures', 'NNS'), (('contrast', 'NN'), ('contrast', 'VBP'))],
 [('report', 'NN'), (('issued', 'VBN'), ('issued', 'VBD'))],
 [('*', '-NONE-'), (('earlier', 'JJR'), ('earlier', 'RBR'))],
 [("'s", 'POS'), (('F.W.', 'CC'), ('F.W.', 'NNP'))],
 [('shipyard', 'NN'), (('that', 'IN'), ('that', 'WDT'))],
 [(',', ','), (('planned', 'VBN'), ('planned', 'VBD'))]]